In [1]:
%cd /Users/tillschaland/Project/Gutemberg-Graph-Network/GutenbergCorpus/

/Users/tillschaland/Project/Gutemberg-Graph-Network/gutenbergCorpus


In [2]:
import pandas as pd
import  os
import definitions
import regex as re
import spacy

/Users/tillschaland/miniconda3/envs/gutenbergDBInit/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from definitions import DATA_DIR
all_books=os.listdir(DATA_DIR+'books/final/')

In [4]:
dfs=[]

for i,file in enumerate(all_books):
    df = pd.read_json(f"./data/books/final/{file}",orient='index')
    dfs.append(df)

In [79]:
def createCharacterInfo(df):
    characterlist=list()
    bookid=None
    for i,row in df.iterrows():
        characterlist.extend(list(row.PERS))
        bookid=row.bookindex

    characterlist=list(set(characterlist))
    temp_=[]
    for i,char in enumerate(characterlist):
        persdict=dict()
        persdict['sentenceBookID']=i
        persdict['name']=char
        persdict['bookID']=bookid
        temp_.append(persdict)
    return temp_

In [96]:
def createBookFixture(dfs):
    fixtures_=[] # 'bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects'
    for i,df in enumerate(dfs):
        series=df.iloc[0]
        fixtures_.append({'pk':i,
                          'id':series.id,
                          'title':series.title,
                          'author':series.author,
                          'authoryearofbirth':series.authoryearofbirth,
                          'authoryearofdeath':series.authoryearofdeath,
                          'language':series.language,
                          'subjects':series.subjects
                          })
        
    return fixtures_

def createSentFixture(df, bookFix, topicFix):
    temp_=[]# sentenceid(pk), bookid, sentenceText,sentIDBook, topicID(pk)
    for i,row in df.iterrows():
        try:
            topic=next(filter(lambda x: x['Topic']==row.Topic and x['Name']==row.Name,topicFix))
        except  TypeError as err:
            print("No Topic Found")            
        try:
            book=next(filter(lambda x: x['id']==row.id,bookFix))
        except TypeError as err:
            print("Noo book found")
        temp_.append(
            {
            'pk':row.sentenceIndex,
            'bookID': book['pk'],
            'sentenceText':row.sents,
            'sentIDBook':i,
            'topicID':topic['pk'],
            }
        )
    return temp_

In [7]:
for df in dfs:
    df.rename({'index':'sentenceIndex'},axis=1,inplace=True)

In [8]:
def extractChar(df):
    allChars = list()
    for i,row in df.iterrows():
        chars_ =row.PERS
        for char in chars_:
            allChars.append(char)
    return list(set(allChars))

In [9]:
def extractTopics(df):
    rel_cols=['bookindex','id','Topic','Name','Top_n_words']
    temp_ = df[rel_cols]
    listoftopics=list()
    for i,row in temp_.iterrows():
        dict_=dict()
        dict_['Topic']=row.Topic
        dict_['Name']=row.Name
        dict_['Top_n_words']=processTopics(row.Top_n_words)
        dict_['bookindex']=row.bookindex
        dict_['id']=row.id
        listoftopics.append(dict_)
    return listoftopics

def processTopics(topnwords):
    list_of_words=topnwords.split('-')
    list_of_words=[word.strip(' ') for word in list_of_words]
    return list_of_words


def removeDup(l):
    new_d = []
    for x in l:
        if x not in new_d:
            new_d.append(x)
    return new_d

def removeDictDuplicates(l):
    seen = set()
    new_l = []
    for d in l:
        t = tuple(d.items())
        if t not in seen:
            seen.add(t)
            new_l.append(d)        
    return sorted(new_l, key=lambda x: x['Topic'])   # sort by topic


In [10]:
characterTable=[]
for df in dfs:
    characterTable.extend(createCharacterInfo(df))

characterTable =  [{'pk':i, 'name':char['name'],'bookID':char['bookID'], 'sentenceBookID':char['sentenceBookID']}for i,char in enumerate(characterTable)]

In [11]:
df.head()

,sentenceIndex,bookindex,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,sents,Document,Topic,Name,Top_n_words,Probability,Representative_document,PERS
0,17952,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,,,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
1,17953,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,[Illustration] The Enormous Room by E. E. Cumm...,[Illustration] The Enormous Room by E. E. Cumm...,3,3_room_enormous_pillars_inhabitants,room - enormous - pillars - inhabitants - eccl...,0.495287,False,[]
2,17954,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,FOUND.” He was lost by the Norton-Harjes Ambul...,FOUND.” He was lost by the Norton-Harjes Ambul...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
3,17955,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,to life—with the help of powerful and willing ...,to life—with the help of powerful and willing ...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
4,17956,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,the Atlantic. The following documents tell the...,the Atlantic. The following documents tell the...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]


In [12]:
t=removeDup(extractTopics(df))
t =  [{'pk':i, 'Topic':topic['Topic'],'Name':topic['Name'], 'top_n_word':topic['Top_n_words'],'bookindex':topic['bookindex']}for i,topic in enumerate(t)]

In [37]:
def createSent2Char(df, dfs_character):

    def helpFilter(charDict, char):
        return charDict['name']==char
    
    sent2char=[]

    for i,row in df.iterrows():
        characters=row.PERS
        for char in characters:
            filteredChar = next(filter(lambda x: helpFilter(x,char), dfs_character))
            charpk = filteredChar['pk']
            sentID = row.sentenceIndex
            sent2char.append({'charID':charpk, 'sentID':sentID})

    return sent2char

In [13]:
def isTopic(id,item):
    """
    Filter Function for filter()
    
    """
    return id==item['Topic']   


In [178]:
dfs_book=[] # 'bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects'
dfs_sentence=[] # sentenceid(pk), bookid, sentenceText,sentIDBook, topicID(pk)
dfs_Sent2Char=[] # charID(sent.pk), sentID(sent.pk)
dfs_character=[] # id(pk),name, bookid(book(pk))
dfs_topic=[] # id(pk), bookid(book.pk), bookInternID(topicID),name, topnwords
for df in dfs:
    #df=df.reset_index().rename({'index':'sentIDBook'},axis=1)
    dfs_book.append(df[['bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects']])
    dfs_sentence.append(df[['sentenceIndex', 'id','sents','Topic']])
    dfs_character.extend(createCharacterInfo(df))
    dfs_topic.extend(removeDup(extractTopics(df)))

characterFixture =  [{'pk':i, 'name':char['name'],'bookID':char['bookID'], 'sentenceBookID':char['sentenceBookID']}for i,char in enumerate(characterTable)]
topicFixture =  [{'pk':i, 'Topic':topic['Topic'],'Name':topic['Name'], 'Top_n_words':topic['Top_n_words'],'bookindex':topic['bookindex']}for i,topic in enumerate(dfs_topic)]
bookFixture=createBookFixture(dfs)
sentenceFixture=[]
sent2charFixture=[]
for df in dfs:
    sent2charFixture.extend(createSent2Char(df, characterFixture))
    fitures=createSentFixture(df,bookFixture,topicFixture)
    sentenceFixture.extend(fitures)
sent2charFixture=[{'pk':i, 'charID':fix['charID'], 'sentID':fix['sentID']} for i,fix in enumerate(sent2charFixture)]

In [181]:
list_of_Sent=list(filter(lambda x: x['fields']['charID']==15, sent2charFixture))

In [190]:
for sent in list_of_Sent:
    toSearch=sent['fields']['sentID']
    sentences=list(filter(lambda x: x['pk']==toSearch,sentenceFixture))

bookid=next(filter(lambda x: x['pk']==15, characterFixture))['fields']['bookID']
allTopics=list(filter(lambda x: x['fields']['bookindex']==bookid, topicFixture))

In [191]:
allTopics

[{'model': 'webDB.Topic',
  'pk': 0,
  'fields': {'TopicID': 35,
   'Name': '35_joscelyn_harry_joscelyns_vol',
   'Top_n_words': ['joscelyn',
    'harry',
    'joscelyns',
    'vol',
    'chapter',
    'mr',
    'marlborough',
    'reserved',
    'blackett',
    'oliphant'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 1,
  'fields': {'TopicID': 4,
   'Name': '4_father_fathers_son_family',
   'Top_n_words': ['father',
    'fathers',
    'son',
    'family',
    'quarrel',
    'sons',
    'terror',
    'passion',
    'number',
    'account'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 2,
  'fields': {'TopicID': 71,
   'Name': '71_days_happen_believing_busier',
   'Top_n_words': ['days',
    'happen',
    'believing',
    'busier',
    'wellinformed',
    'timeit',
    'review',
    'informationthe',
    'oldhad',
    'novel'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 3,
  'fields': {'TopicID': -1,
   'Name': '-1_like_little_did_man',
   'Top_n_words':

In [192]:

distr_=dict()
for topic in allTopics:
    distr_[topic['fields']['TopicID']]= {'TopicName:':topic['fields']['Name'], 'Count':0}
for sent in sentences:
    distr_[sent['fields']['topicID']]['Count']+=1


print(distr_)

{35: {'TopicName:': '35_joscelyn_harry_joscelyns_vol', 'Count': 0}, 4: {'TopicName:': '4_father_fathers_son_family', 'Count': 0}, 71: {'TopicName:': '71_days_happen_believing_busier', 'Count': 0}, -1: {'TopicName:': '-1_like_little_did_man', 'Count': 0}, 21: {'TopicName:': '21_mother_mothers_partly_cleverness', 'Count': 0}, 53: {'TopicName:': '53_knitting_knit_yarn_knittingand', 'Count': 0}, 5: {'TopicName:': '5_light_moonlight_lights_lamps', 'Count': 0}, 28: {'TopicName:': '28_sound_sounds_noise_voices', 'Count': 0}, 38: {'TopicName:': '38_glow_stretched_burning_table', 'Count': 0}, 22: {'TopicName:': '22_parlour_chairs_mending_furniture', 'Count': 0}, 52: {'TopicName:': '52_land_country_folk_lands', 'Count': 0}, 37: {'TopicName:': '37_daughter_thirty_vet_young', 'Count': 0}, 69: {'TopicName:': '69_qualities_blessedness_orderly_chivalrous', 'Count': 0}, 0: {'TopicName:': '0_joan_joans_said_miss', 'Count': 0}, 12: {'TopicName:': '12_bed_sleep_tears_pillow', 'Count': 0}, 29: {'TopicName

In [179]:
characterFixture=createFinalCharacterFixtur(characterFixture)
topicFixture=createFinalTopicFixtur(topicFixture)
bookFixture=createFinalBookFixtur(bookFixture)
sentenceFixture=createFinalSentenceFixtur(sentenceFixture)
sent2charFixture=createFinalSent2CharFixtur(sent2charFixture)

In [180]:
topicFixture

[{'model': 'webDB.Topic',
  'pk': 0,
  'fields': {'TopicID': 35,
   'Name': '35_joscelyn_harry_joscelyns_vol',
   'Top_n_words': ['joscelyn',
    'harry',
    'joscelyns',
    'vol',
    'chapter',
    'mr',
    'marlborough',
    'reserved',
    'blackett',
    'oliphant'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 1,
  'fields': {'TopicID': 4,
   'Name': '4_father_fathers_son_family',
   'Top_n_words': ['father',
    'fathers',
    'son',
    'family',
    'quarrel',
    'sons',
    'terror',
    'passion',
    'number',
    'account'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 2,
  'fields': {'TopicID': 71,
   'Name': '71_days_happen_believing_busier',
   'Top_n_words': ['days',
    'happen',
    'believing',
    'busier',
    'wellinformed',
    'timeit',
    'review',
    'informationthe',
    'oldhad',
    'novel'],
   'bookindex': 2}},
 {'model': 'webDB.Topic',
  'pk': 3,
  'fields': {'TopicID': -1,
   'Name': '-1_like_little_did_man',
   'Top_n_words':

In [173]:
def createFinalCharacterFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Character",
            "pk":fix['pk'],
            "fields": {
                'Name':fix['name'],
                'bookID':fix['bookID'],
            }
            }
        )
    return temp_

In [174]:
def createFinalTopicFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Topic",
            "pk":fix['pk'],
            "fields": {
                'TopicID':fix['Topic'],
                'Name':fix['Name'],
                'Top_n_words':fix['Top_n_words'],
                'bookindex':fix['bookindex']
            }
            }
        )
    return temp_

In [175]:
def createFinalSent2CharFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Sent2Char",
            "pk":fix['pk'],
            "fields": {
                'charID':fix['charID'],
                'sentID':fix['sentID']
            }
            }
        )
    return temp_

In [176]:
def createFinalSentenceFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Sentence",
            "pk":fix['pk'],
            "fields": {
                'bookID':fix['bookID'],
                'sentenceText':fix['sentenceText'],
                'sentIDBook':fix['sentIDBook'],
                'topicID':fix['topicID']
            }
            }
        )
    return temp_

In [177]:
def createFinalBookFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Book",
            "pk":fix['pk'],
            "fields": {
                "id":fix['id'],
                "title":fix['title'],
                "author":fix['author'],
                "authoryearofbirth":int(fix['authoryearofbirth']),
                "authoryearofdeath":int(fix['authoryearofdeath']),
                "language":fix['language'],
                "subjects":fix['subjects'],
            }
            }
        )
    return temp_

In [148]:
import json
fixtureFileNames=['character','topic','book','sentence','sent2char']
fixturesCombined=[characterFixture,topicFixture,bookFixture,sentenceFixture,sent2charFixture]
for fixture, filename in zip(fixturesCombined,fixtureFileNames):
    with open(f"./data/Fixtures_small/{filename}FixturesSmall.json", 'w') as f:
        json.dump(fixture,f)


# OLD

In [ ]:
def createSentenceFixture(df):
    result=list()
    for i,row in df.iterrows():
        sentIndex=i
        model = "webDB.Sentence"
        sentenceText=row['sents']
        pk=row['sentenceIndex']
        bookID = row['id']
        entry={
            "model": model,
            "pk":pk,
            "fields": {
            "bookID": bookID,
            "sentenceText":sentenceText,
            "sentenceIndex": sentIndex
            }
            }
        result.append(entry)
    return result

In [83]:
def createBookFixture(df,pk):
    bookID=df.iloc[0]['id']
    model = "webDB.Book"
    bookname=df.iloc[0]['title']
    bookauthor=df.iloc[0]['author']

    entry={
        "model": model,
        "pk":pk,
        "fields": {
        "bookID": bookID,
        "bookName": bookname,
        "bookauthor":bookauthor
        }
        }

    return entry

In [148]:
Bookfixtures=[]
Sentencefitures=[]
Characterfixtures=[]
Topicfixtures=[]
for i, df in enumerate(dfs):
    Bookfixtures.append(createBookFixture(df,i))
    Sentencefitures.extend(createSentenceFixture(df))

In [150]:
import json

In [151]:
with open("BookFixtures.json", "w") as final:
   json.dump(Bookfixtures, final)
with open("SentenceFixtures.json", "w") as final:
   json.dump(Sentencefitures, final)

# One  Table Fixture

In [110]:
fixturetest=[]
for i,row in df.iterrows():
    pk = row['sentIndex']
    bookid=row['id']
    authorname=row['author']
    sent=row['sents']
    topics=row['probs']
    pers=row['PERS']
    
    entry={
        "model":"webDB.fulldata",
        "pk":pk,
        "fields":{
        "bookID":bookid,
        "authorname":authorname,
        "sent":sent,
        "topics":topics,
        "pers":pers
        }
    }
    fixturetest.append(entry)

In [112]:
import json
with open("testFixture.json", "w") as final:
   json.dump(fixturetest, final)

# Global Fixture

In [94]:
sentenceID=0
topicID=0
characterID=0
Globalfixture=[]
for i,file in enumerate(all_books):
    bookid=i
    df = pd.read_json(f"./data/books/{file}",orient='index')
    df=df.reset_index().drop('index',axis=1)
    df=df.drop('sentIndex',axis=1).reset_index().rename({'index':'sentIndex'},axis=1)
    Globalfixture.append(createBookFixture(df,bookid))
    Globalfixture.extend(createSentenceFixture(df,sentenceID))
    
    

In [93]:
def createSentenceFixture(row,sentCount):
    result=list()
    sentCount=sentCount
    for i,row in df.iterrows():
        sentIndex=row['sentIndex']
        model = "webDB.Sentence"
        sentenceText=row['sents']
        bookID = row['id']
        topicID=row['probs']
        entry={
            "model": model,
            "pk":sentCount,
            "fields": {
            "bookID": bookID,
            "sentenceText":sentenceText,
            "sentenceIndex": sentIndex,
            "topicID":topicID,
            }
            }
        sentCount+=1
        result.append(entry)
    return result

In [95]:
Globalfixture

[{'model': 'webDB.Book',
  'pk': 0,
  'fields': {'bookID': 'PG5633',
   'bookName': 'The Martial Adventures of Henry and Me',
   'bookauthor': 'White, William Allen'}},
 {'model': 'webDB.Sentence',
  'pk': 0,
  'fields': {'bookID': 'PG5633',
   'sentenceText': ' THE MARTIAL ADVENTURES OF HENRY AND ME BY WILLIAM ALLEN WHITE Author of "A Certain Rich Man," etc. WITH ILLUSTRATIONS BY TONY SARG CONTENTS CHAPTER I IN WHICH WE BEGIN OUR SENTIMENTAL JOURNEY II IN WHICH WE OBSERVE THE "ROCKET\'S RED GLARE" III IN WHICH WE ENCOUNTER "BOMBS BURSTING IN AIR" IV WHEREIN WE FIND THAT "OUR FLAG IS STILL THERE" V IN WHICH WE DISCERN THINGS "BY THE DAWN\'S EARLY LIGHT" VI WHEREIN WE BECOME A TRIO AND JOURNEY TO ITALY VII WHEREIN WE CONSIDER THE WOMAN PROPOSITION VIII IN WHICH WE DISCOVER "A NEW HEAVEN AND A NEW EARTH" IX IN WHICH WE RETURN TO "THE LAND OF THE FREE" ILLUSTRATIONS Frontispiece And at that it seems a lot of money to pay for a rig which can be',
   'sentenceIndex': 0,
   'topicID': 0}},
 

In [ ]:
def createSent2TopicFixture(df,pk):
    result=list()
    sentCount=pk
    for i,row in df.iterrows():
        sentIndex=row['sentIndex']
        model = "webDB.Topic"
        sentenceText=row['sents']
        topicID = row['probs']
        entry={
            "model": model,
            "pk":sentCount,
            "fields": {
            "TopicID": topicID,
            "sentenceIndex": sentIndex,
            "bookID":bookID
            }
            }
        sentCount+=1
        result.append(entry)
    return result

In [51]:
def createCharacterFixture(df):
    result=list()
    for i,row in df.iterrows():
        sentIndex=row['sentIndex']
        model = "webDB.Character"
        sentenceText=row['sents']
        bookID = row['id']

        entry={
            "model": model,
            "pk":i,
            "fields": {
            "bookID": bookID,
            "sentenceText":sentenceText,
            "sentenceIndex": sentIndex,
            "bookID":bookID
            }
            }
        result.append(entry)
    return result